In [1]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

#Creates a blank canvas (black image) with dimensions 720x1280 pixels and 3 color channels (BGR). 
#The dtype=np.uint8 ensures pixel values are 8-bit integers (0–255), suitable for images.
canvas = np.zeros((720, 1280, 3), dtype=np.uint8) 

# Drawing settings
draw_color = (255, 0, 255)  #Sets the default drawing color to purple in BGR format
#(Blue=255, Green=0, Red=255)
brush_thickness = 15 #Sets the thickness of the drawing brush (15 pixels)
eraser_thickness = 50 #Sets the thickness of the eraser (50 pixels).
xp, yp = 0, 0 #Initializes the previous x, y coordinates
#for drawing continuity (used to draw lines between points).

# Creates a black header image (90 pixels tall, 1280 pixels wide) to display instructions.
header = np.zeros((90, 1280, 3), dtype=np.uint8)
cv2.putText(header, "1: Purple   2: Blue   3: Green   E: Eraser", (20, 35),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(header, "C: Clear   Q: Quit", (20, 75),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img[0:90, 0:1280] = header  # this code takes the top `90` pixels of the image (`img`)
    #and replaces them with whatever is stored in `header`.
    #This is commonly used for adding a title bar or overlay to an image.



    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            lm_list = []
            h, w, _ = img.shape
            for id, lm in enumerate(hand_landmarks.landmark):
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append((cx, cy))

            if lm_list:
                x1, y1 = lm_list[8]   # Index tip
                x2, y2 = lm_list[12]  # Middle tip

                # Drawing Mode: Only Index Up
                if y1 < lm_list[6][1] and y2 > lm_list[10][1]: #comparison between the y coordinates
                    # for checking finer is up or down
                    cv2.circle(img, (x1, y1), 15, draw_color, cv2.FILLED)
                    if xp == 0 and yp == 0:
                        xp, yp = x1, y1

                    thickness = eraser_thickness if draw_color == (0, 0, 0) else brush_thickness
                    cv2.line(img, (xp, yp), (x1, y1), draw_color, thickness)
                    cv2.line(canvas, (xp, yp), (x1, y1), draw_color, thickness)
                    xp, yp = x1, y1

                else:
                    xp, yp = 0, 0

            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            #Uses MediaPipe's drawing utility to draw landmarks (dots) and connections (lines) on the detected hand, 
            #overlaying them on the webcam frame for visualization

    # Merge canvas with webcam
    img_gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY) #Converts the canvas to grayscale for thresholding.
    #Thresholding is a technique used to separate certain pixel values—often based on
    #brightness—to create a binary mask
# What is binary mask??
    #A binary mask is a type of image processing technique where each pixel is classified into two categories—either\
    #black (0) or white (255)—based on a condition.
    # In a binary mask, we are selectively keeping a particular portion of the image while masking (ignoring) the rest by making it black.
#How It Works
#- Target Selection → We identify the pixels we want to keep (based on brightness, color, or another condition).
#- Masking the Rest → Pixels that don’t match the condition are turned black (0), effectively removing or ignoring them.
#- Applying the Mask → The mask can then be used to overlay or extract specific regions.

    _, inv_mask = cv2.threshold(img_gray, 50, 255, cv2.THRESH_BINARY_INV)
    inv_mask = cv2.cvtColor(inv_mask, cv2.COLOR_GRAY2BGR) #Converts the grayscale mask to BGR to match the webcam frame's format.
    img = cv2.bitwise_and(img, inv_mask) #bitwise AND (cv2.bitwise_and()) is commonly used in binary masking to isolate specific
#parts of an image, based on a mask. 
    img = cv2.bitwise_or(img, canvas) #- bitwise_and() keeps only overlapping areas where both images have white pixels.
#- bitwise_or() merges both images, ensuring at least one contributes to the final output.


    cv2.imshow("Virtual Painter", img)

    key = cv2.waitKey(1)
    if key == ord('1'):
        draw_color = (255, 0, 255)  # Purple
    elif key == ord('2'):
        draw_color = (255, 0, 0)  # Blue
    elif key == ord('3'):
        draw_color = (0, 255, 0)  # Green
    elif key == ord('e'):
        draw_color = (0, 0, 0)  # Eraser
    elif key == ord('c'):
        canvas[:] = 0  # Clear canvas
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# How Thresholding works?

- Convert to Grayscale: First, your image is turned into a grayscale format, where brightness values range from 0 (black) to 255 (white).
- Apply a Threshold Value: You choose a threshold value (e.g., 50). Pixels brighter than this value will be assigned white (255), and pixels darker will be assigned black (0).
- Binary Mask is Created: Now, your image is simplified into only two colors (black and white), forming a binary mask.

# Key Difference b/w Masking and thresholding
- Thresholding is the process of converting an image into black-and-white based on pixel intensity.
- A Binary Mask is the final output, often used for further image processing like blending, segmentation, or object detection


# Normal vs. Inverse Thresholding:
- Normal Threshold (cv2.THRESH_BINARY): Pixels above the threshold become white, others remain black.
- Inverse Threshold (cv2.THRESH_BINARY_INV): Pixels above the threshold become black, others turn white.


# Why syntax is like this?

- _, inv_mask = ...
- The underscore (_) is a throwaway variable capturing the threshold function’s first output (which we don’t need).
- inv_mask is the actual binary mask we want—this stores the processed image where pixels are either black (0) or white (255).
Why is _ Used?
- The cv2.threshold() function returns two values:
- The threshold value used (which we don’t need here).
- The processed binary mask (which we do need).
- _ is commonly used to ignore unnecessary output in Python.

